# Interdisciplinary Health Data Competition - Data Cleaning

## Import necessary libraries

In [2]:
import pandas as pd
import numpy as np
import warnings

## Agenda

Step 1 - Read in Data Files
- Read in drug and prescription files
- Inspect their initial format
- Inspect their initial data types
- Inspect data distribution

Step 2 - Check for Nulls
- Check for nulls by count
- Check for nulls by percentage
- Replace any known nulls
- Drop columns with ~20% or more missing values
- Drop rows with ~10% or less missing values

Step 3 - Try to Impute Nulls in Percent Change
- analyze head of dataframe
- analyze the cost per script rank
- remove original percent change features

Step 4 - Properly Join the Two Files
- join prescription 2012 to prescription 2016
- create new percent change features
- fill in calculations for the percent change columns in prescription
- join drug 2012 to drug 2016

Step 5 - Create Lists Where Applicable

Step 6 - Write cleaned data to new files

## Step 1 - Read in Data Files

In [5]:
# read in the first file and inspect
drug = pd.read_excel("DrugDetailMerged.xlsx")
drug.head()

,YEAR,RECORD_TYPE,NUMBER_SCRIPTS,NDC9,PRODUCT_NDC,LAUNCH_YEAR,NPROPNAME,PROPNAME,GENERIC_DRUG_IND_CODE,THER_CLASS,...,INSURER_PAID_STDDEV,INSURER_PAID_SUM,MEMBER_PAID_MEDIAN,MEMBER_PAID_MEAN,MEMBER_PAID_STDDEV,MEMBER_PAID_SUM,TOTAL_PAID_MEDIAN,TOTAL_PAID_MEAN,TOTAL_PAID_STDDEV,TOTAL_PAID_SUM
0,2012,NDC,61,449461023,44946-1023,2011.0,".ALPHA.,TOCOPHEROL ACETATE, DL,, ASCORBIC ACID...",MULTIVITAMIN WITH FLUORIDE,Generic,Nutritional Products,...,6.773281,321.68,15.23,12.211967,7.758228,744.93,17.44,17.485410,6.349664,1066.61
1,2012,NDC,52,449461024,44946-1024,2011.0,".ALPHA.,TOCOPHEROL ACETATE, DL,, ASCORBIC ACID...",MULTIVITAMIN WITH FLUORIDE,Generic,Nutritional Products,...,6.621973,160.28,10.18,11.648654,9.022594,605.73,14.96,14.730962,9.387626,766.01
2,2012,NDC,24,449461025,44946-1025,2011.0,".ALPHA.,TOCOPHEROL ACETATE, DL,, ASCORBIC ACID...",MULTIVITAMIN WITH FLUORIDE,Generic,Nutritional Products,...,6.543839,109.56,10.00,11.942917,5.914719,286.63,17.32,16.507917,5.275046,396.19
3,2012,NDC,25,449461024,44946-1024,2011.0,".ALPHA.,TOCOPHEROL ACETATE, DL,, ASCORBIC ACID...",MULTIVITAMIN WITH FLUORIDE,Generic,Nutritional Products,...,2.962455,218.90,0.00,0.000000,0.000000,0.00,7.37,8.761600,2.959877,219.04
4,2012,NDC,36,758540301,75854-301,2012.0,".BETA.,CAROTENE, ASCORBIC ACID, CHOLECALCIFERO...",PRENATE ELITE,Brand,Nutritional Products,...,81.140106,4127.63,40.61,63.982222,79.410866,2303.36,153.85,178.638611,106.344757,6430.99


In [6]:
# inspect data types
drug.dtypes

YEAR                         int64
RECORD_TYPE                 object
NUMBER_SCRIPTS               int64
NDC9                         int64
PRODUCT_NDC                 object
LAUNCH_YEAR                float64
NPROPNAME                   object
PROPNAME                    object
GENERIC_DRUG_IND_CODE       object
THER_CLASS                  object
LABELERNAME                 object
DOSAGE_FORM                 object
ACTIVE_STRENGTH             object
ACTIVE_STRENGTH_UNIT        object
PAYER                       object
UNIQUE_MEMBERS               int64
TOTAL_COST                 float64
COST_PER_USER              float64
COST_PER_SCRIPT            float64
COST_PER_DAYS_SUPPLY       float64
COST_PER_UNIT_DISPENSED    float64
TOTAL_SCRIPTS_FILLED         int64
DAYS_SUPPLY_MEDIAN         float64
DAYS_SUPPLY_MEAN           float64
DAYS_SUPPLY_STDDEV         float64
QUANTITY_MEDIAN            float64
QUANTITY_MEAN              float64
QUANTITY_STDDEV            float64
INSURER_PAID_MEDIAN 

In [7]:
# Inspect data distribution
drug.describe()

,YEAR,NUMBER_SCRIPTS,NDC9,LAUNCH_YEAR,UNIQUE_MEMBERS,TOTAL_COST,COST_PER_USER,COST_PER_SCRIPT,COST_PER_DAYS_SUPPLY,COST_PER_UNIT_DISPENSED,...,INSURER_PAID_STDDEV,INSURER_PAID_SUM,MEMBER_PAID_MEDIAN,MEMBER_PAID_MEAN,MEMBER_PAID_STDDEV,MEMBER_PAID_SUM,TOTAL_PAID_MEDIAN,TOTAL_PAID_MEAN,TOTAL_PAID_STDDEV,TOTAL_PAID_SUM
count,58299.000000,58299.000000,5.829900e+04,51822.000000,58299.000000,5.829900e+04,58299.000000,58299.000000,58299.000000,58299.000000,...,58299.000000,5.829900e+04,58299.000000,58299.000000,58299.000000,5.829900e+04,58299.000000,58299.000000,58299.000000,5.829900e+04
mean,2014.117704,1907.544160,2.925065e+08,2006.495330,761.104822,1.710579e+05,797.708650,208.245714,7.390627,17.368790,...,121.661856,1.454995e+05,7.939869,16.366089,27.944861,2.438385e+04,179.519354,208.245714,123.413276,1.710579e+05
std,1.996551,6389.540882,2.825932e+08,8.628465,2783.494766,1.315084e+06,5054.301835,1131.587113,51.156846,231.158146,...,625.345322,1.206305e+06,25.826820,47.488616,96.090746,1.447407e+05,1027.125319,1131.587113,626.286942,1.315084e+06
min,2012.000000,11.000000,0.000000e+00,1900.000000,11.000000,2.890000e+00,0.180625,0.180625,0.008600,0.001492,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.190000,0.180625,0.000000,2.890000e+00
25%,2012.000000,71.000000,3.952838e+06,2004.000000,30.000000,2.207805e+03,24.401119,10.773499,0.356612,0.182733,...,7.366863,1.560860e+03,0.000000,0.996496,1.907238,1.800100e+02,8.120000,10.773499,8.205806,2.207805e+03
50%,2016.000000,248.000000,2.315505e+08,2009.000000,98.000000,1.204032e+04,70.288785,29.999088,0.983575,0.529266,...,23.313743,8.773700e+03,1.910000,5.221292,7.078508,1.408240e+03,22.460000,29.999088,24.652300,1.204032e+04
75%,2016.000000,1118.500000,5.789605e+08,2011.000000,435.000000,5.769987e+04,257.863950,103.254048,3.490333,2.229693,...,71.973515,4.246544e+04,10.000000,15.259937,21.156922,8.706125e+03,81.370000,103.254048,74.584304,5.769987e+04
max,2016.000000,222810.000000,9.920705e+08,2019.000000,122975.000000,1.450619e+08,375453.225000,73004.793750,7007.324345,17589.369406,...,51937.267544,1.367770e+08,1783.840000,2833.294687,4975.867439,9.728195e+06,54247.490000,73004.793750,52087.073348,1.450619e+08


In [9]:
# read in the first file and inspect
prescription = pd.read_excel("SummaryMerged.xlsx")
prescription.head()

,YEAR,RECORD_TYPE,NUMBER_SCRIPTS,NPROPNAME,THER_CLASS,PAYER,UNIQUE_MEMBERS,TOTAL_COST,COST_PER_USER,COST_PER_SCRIPT,...,PCT_SCRIPTS_0_18,PCT_SCRIPTS_19_44,PCT_SCRIPTS_45_64,PCT_SCRIPTS_65_PLUS,PCT_SCRIPTS_FEMALE,PCT_SCRIPTS_MALE,PCT_URBAN_CORE,PCT_SUBURBAN,PCT_MICROPOLITAN,PCT_RURAL_SMALLTOWN
0,2012,Drug,137,".ALPHA.,TOCOPHEROL ACETATE, DL,, ASCORBIC ACID...",Nutritional Products,Commercial,99,2228.81,22.513232,16.268686,...,100.0,0.0,0.0,0.0,48.905109,51.094891,20.437956,21.167883,45.255474,13.138686
1,2012,Drug,25,".ALPHA.,TOCOPHEROL ACETATE, DL,, ASCORBIC ACID...",Nutritional Products,Minnesota Health Care Programs,13,219.04,16.849231,8.761600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012,Drug,36,".BETA.,CAROTENE, ASCORBIC ACID, CHOLECALCIFERO...",Nutritional Products,Commercial,26,6430.99,247.345769,178.638611,...,0.0,100.0,0.0,0.0,100.000000,0.000000,NaN,NaN,NaN,NaN
3,2012,Drug,210,".BETA.,CAROTENE, ASCORBIC ACID, CHOLECALCIFERO...",Nutritional Products,Commercial,92,3810.43,41.417717,18.144905,...,NaN,NaN,NaN,NaN,100.000000,0.000000,NaN,NaN,NaN,NaN
4,2012,Drug,117,".BETA.,CAROTENE, ASCORBIC ACID, CHOLECALCIFERO...",Nutritional Products,Minnesota Health Care Programs,56,1082.49,19.330179,9.252051,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# inspect data types
prescription.dtypes

YEAR                                 int64
RECORD_TYPE                         object
NUMBER_SCRIPTS                       int64
NPROPNAME                           object
THER_CLASS                          object
PAYER                               object
UNIQUE_MEMBERS                       int64
TOTAL_COST                         float64
COST_PER_USER                      float64
COST_PER_SCRIPT                    float64
COST_PER_DAYS_SUPPLY               float64
COST_PER_UNIT_DISPENSED            float64
TOTAL_SCRIPTS_FILLED                 int64
PCT_CHANGE_COST_PER_SCRIPT         float64
TOTAL_COST_RANK                      int64
COST_PER_USER_RANK                   int64
COST_PER_SCRIPT_RANK                 int64
COST_PER_DAYS_SUPPLY_RANK            int64
COST_PER_UNIT_DISPENSED_RANK         int64
TOTAL_SCRIPTS_FILLED_RANK            int64
PCT_CHANGE_COST_PER_SCRIPT_RANK    float64
DAYS_SUPPLY_MEDIAN                 float64
DAYS_SUPPLY_MEAN                   float64
DAYS_SUPPLY

In [11]:
# Inspect data distribution
prescription.describe()

,YEAR,NUMBER_SCRIPTS,UNIQUE_MEMBERS,TOTAL_COST,COST_PER_USER,COST_PER_SCRIPT,COST_PER_DAYS_SUPPLY,COST_PER_UNIT_DISPENSED,TOTAL_SCRIPTS_FILLED,PCT_CHANGE_COST_PER_SCRIPT,...,PCT_SCRIPTS_0_18,PCT_SCRIPTS_19_44,PCT_SCRIPTS_45_64,PCT_SCRIPTS_65_PLUS,PCT_SCRIPTS_FEMALE,PCT_SCRIPTS_MALE,PCT_URBAN_CORE,PCT_SUBURBAN,PCT_MICROPOLITAN,PCT_RURAL_SMALLTOWN
count,8455.000000,8455.000000,8455.000000,8.455000e+03,8455.000000,8455.000000,8455.000000,8455.000000,8455.000000,7004.000000,...,5210.000000,5210.000000,5210.000000,5210.000000,7724.000000,7724.000000,6028.000000,6028.000000,6028.000000,6028.000000
mean,2014.019160,13152.917445,4308.200591,1.179480e+06,2578.153297,569.442394,21.229498,51.895639,13152.917445,45.063242,...,9.209697,23.696248,36.997350,30.096706,59.289551,40.710449,60.939713,9.934860,11.746726,17.378701
std,2.000026,46422.933559,15403.472815,4.776704e+06,10931.001135,2447.865128,121.156683,427.740200,46422.933559,305.890815,...,18.814400,20.574852,23.320994,34.092183,18.204456,18.204456,12.747692,3.903336,5.173913,8.409159
min,2012.000000,11.000000,11.000000,2.998000e+01,1.177277,1.104720,0.027031,0.002460,11.000000,-99.662323,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2012.000000,136.000000,51.000000,8.977245e+03,39.691584,16.907883,0.621009,0.269303,136.000000,-27.329524,...,0.000000,6.238080,16.189259,2.658529,49.217004,30.825780,54.024815,7.770706,8.695734,11.764706
50%,2016.000000,803.000000,272.000000,7.927720e+04,145.470675,55.666024,2.058153,1.144981,803.000000,0.207009,...,0.529011,20.563030,38.206872,12.519637,59.286234,40.713766,61.341711,9.803353,11.411118,16.307281
75%,2016.000000,5487.500000,1871.500000,5.507450e+05,718.516435,224.981117,8.330387,6.779317,5487.500000,52.000604,...,8.646617,36.000743,55.049245,59.614436,69.174220,50.782996,68.702872,11.902961,14.204316,21.686767
max,2016.000000,790872.000000,381363.000000,1.775626e+08,238221.929375,57317.515000,7007.324345,15867.378285,790872.000000,7613.114958,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,39.333333,60.937500,81.699346


## Step 2 - Check for Nulls

In [12]:
# count of nulls
drug.isnull().sum()

YEAR                          0
RECORD_TYPE                   0
NUMBER_SCRIPTS                0
NDC9                          0
PRODUCT_NDC                6477
LAUNCH_YEAR                6477
NPROPNAME                     0
PROPNAME                    185
GENERIC_DRUG_IND_CODE         0
THER_CLASS                    0
LABELERNAME                6024
DOSAGE_FORM                3178
ACTIVE_STRENGTH            4460
ACTIVE_STRENGTH_UNIT       4472
PAYER                         0
UNIQUE_MEMBERS                0
TOTAL_COST                    0
COST_PER_USER                 0
COST_PER_SCRIPT               0
COST_PER_DAYS_SUPPLY          0
COST_PER_UNIT_DISPENSED       0
TOTAL_SCRIPTS_FILLED          0
DAYS_SUPPLY_MEDIAN            0
DAYS_SUPPLY_MEAN              0
DAYS_SUPPLY_STDDEV            0
QUANTITY_MEDIAN               0
QUANTITY_MEAN                 0
QUANTITY_STDDEV               0
INSURER_PAID_MEDIAN           0
INSURER_PAID_MEAN             0
INSURER_PAID_STDDEV           0
INSURER_

In [13]:
# Percentage of null values for easier analysis
drug.isnull().sum()* 100 / len(drug)

YEAR                        0.000000
RECORD_TYPE                 0.000000
NUMBER_SCRIPTS              0.000000
NDC9                        0.000000
PRODUCT_NDC                11.109968
LAUNCH_YEAR                11.109968
NPROPNAME                   0.000000
PROPNAME                    0.317330
GENERIC_DRUG_IND_CODE       0.000000
THER_CLASS                  0.000000
LABELERNAME                10.332939
DOSAGE_FORM                 5.451208
ACTIVE_STRENGTH             7.650217
ACTIVE_STRENGTH_UNIT        7.670801
PAYER                       0.000000
UNIQUE_MEMBERS              0.000000
TOTAL_COST                  0.000000
COST_PER_USER               0.000000
COST_PER_SCRIPT             0.000000
COST_PER_DAYS_SUPPLY        0.000000
COST_PER_UNIT_DISPENSED     0.000000
TOTAL_SCRIPTS_FILLED        0.000000
DAYS_SUPPLY_MEDIAN          0.000000
DAYS_SUPPLY_MEAN            0.000000
DAYS_SUPPLY_STDDEV          0.000000
QUANTITY_MEDIAN             0.000000
QUANTITY_MEAN               0.000000
Q

In [14]:
# count of nulls
prescription.isnull().sum()

YEAR                                  0
RECORD_TYPE                           0
NUMBER_SCRIPTS                        0
NPROPNAME                             0
THER_CLASS                            0
PAYER                                 0
UNIQUE_MEMBERS                        0
TOTAL_COST                            0
COST_PER_USER                         0
COST_PER_SCRIPT                       0
COST_PER_DAYS_SUPPLY                  0
COST_PER_UNIT_DISPENSED               0
TOTAL_SCRIPTS_FILLED                  0
PCT_CHANGE_COST_PER_SCRIPT         1451
TOTAL_COST_RANK                       0
COST_PER_USER_RANK                    0
COST_PER_SCRIPT_RANK                  0
COST_PER_DAYS_SUPPLY_RANK             0
COST_PER_UNIT_DISPENSED_RANK          0
TOTAL_SCRIPTS_FILLED_RANK             0
PCT_CHANGE_COST_PER_SCRIPT_RANK    1451
DAYS_SUPPLY_MEDIAN                    0
DAYS_SUPPLY_MEAN                      0
DAYS_SUPPLY_STDDEV                    0
QUANTITY_MEDIAN                       0


In [15]:
# Percentage of null values for easier analysis
prescription.isnull().sum()* 100 / len(prescription)

YEAR                                0.000000
RECORD_TYPE                         0.000000
NUMBER_SCRIPTS                      0.000000
NPROPNAME                           0.000000
THER_CLASS                          0.000000
PAYER                               0.000000
UNIQUE_MEMBERS                      0.000000
TOTAL_COST                          0.000000
COST_PER_USER                       0.000000
COST_PER_SCRIPT                     0.000000
COST_PER_DAYS_SUPPLY                0.000000
COST_PER_UNIT_DISPENSED             0.000000
TOTAL_SCRIPTS_FILLED                0.000000
PCT_CHANGE_COST_PER_SCRIPT         17.161443
TOTAL_COST_RANK                     0.000000
COST_PER_USER_RANK                  0.000000
COST_PER_SCRIPT_RANK                0.000000
COST_PER_DAYS_SUPPLY_RANK           0.000000
COST_PER_UNIT_DISPENSED_RANK        0.000000
TOTAL_SCRIPTS_FILLED_RANK           0.000000
PCT_CHANGE_COST_PER_SCRIPT_RANK    17.161443
DAYS_SUPPLY_MEDIAN                  0.000000
DAYS_SUPPL

In [38]:
# Replace any nulls with known values
# 'PROPNAME' NA means generic
drug['PROPNAME'] = drug['PROPNAME'].fillna('GENERIC')
drug.isnull().sum()* 100 / len(drug)

YEAR                        0.000000
RECORD_TYPE                 0.000000
NUMBER_SCRIPTS              0.000000
NDC9                        0.000000
PRODUCT_NDC                11.109968
LAUNCH_YEAR                11.109968
NPROPNAME                   0.000000
PROPNAME                    0.000000
GENERIC_DRUG_IND_CODE       0.000000
THER_CLASS                  0.000000
LABELERNAME                10.332939
DOSAGE_FORM                 5.451208
ACTIVE_STRENGTH             7.650217
ACTIVE_STRENGTH_UNIT        7.670801
PAYER                       0.000000
UNIQUE_MEMBERS              0.000000
TOTAL_COST                  0.000000
COST_PER_USER               0.000000
COST_PER_SCRIPT             0.000000
COST_PER_DAYS_SUPPLY        0.000000
COST_PER_UNIT_DISPENSED     0.000000
TOTAL_SCRIPTS_FILLED        0.000000
DAYS_SUPPLY_MEDIAN          0.000000
DAYS_SUPPLY_MEAN            0.000000
DAYS_SUPPLY_STDDEV          0.000000
QUANTITY_MEDIAN             0.000000
QUANTITY_MEAN               0.000000
Q

In [39]:
# Drop columns with ~20% of more missing values
#prescription = prescription.drop(['PCT_SCRIPTS_0_18','PCT_SCRIPTS_19_44', 'PCT_SCRIPTS_45_64',
#                                  'PCT_SCRIPTS_65_PLUS', 'PCT_URBAN_CORE', 'PCT_SUBURBAN',
#                                  'PCT_MICROPOLITAN', 'PCT_RURAL_SMALLTOWN'], axis = 1)

In [16]:
# Confirm columns were dropped
prescription.isnull().sum()* 100 / len(prescription)

YEAR                                0.000000
RECORD_TYPE                         0.000000
NUMBER_SCRIPTS                      0.000000
NPROPNAME                           0.000000
THER_CLASS                          0.000000
PAYER                               0.000000
UNIQUE_MEMBERS                      0.000000
TOTAL_COST                          0.000000
COST_PER_USER                       0.000000
COST_PER_SCRIPT                     0.000000
COST_PER_DAYS_SUPPLY                0.000000
COST_PER_UNIT_DISPENSED             0.000000
TOTAL_SCRIPTS_FILLED                0.000000
PCT_CHANGE_COST_PER_SCRIPT         17.161443
TOTAL_COST_RANK                     0.000000
COST_PER_USER_RANK                  0.000000
COST_PER_SCRIPT_RANK                0.000000
COST_PER_DAYS_SUPPLY_RANK           0.000000
COST_PER_UNIT_DISPENSED_RANK        0.000000
TOTAL_SCRIPTS_FILLED_RANK           0.000000
PCT_CHANGE_COST_PER_SCRIPT_RANK    17.161443
DAYS_SUPPLY_MEDIAN                  0.000000
DAYS_SUPPL

In [41]:
# Drop rows with ~10% or less missing values and confirm rows were dropped
#prescription = prescription.dropna(axis=0, subset=['PCT_SCRIPTS_FEMALE', 'PCT_SCRIPTS_MALE'])
#prescription.isnull().sum()* 100 / len(prescription)

YEAR                                0.000000
RECORD_TYPE                         0.000000
NUMBER_SCRIPTS                      0.000000
NPROPNAME                           0.000000
THER_CLASS                          0.000000
PAYER                               0.000000
UNIQUE_MEMBERS                      0.000000
TOTAL_COST                          0.000000
COST_PER_USER                       0.000000
COST_PER_SCRIPT                     0.000000
COST_PER_DAYS_SUPPLY                0.000000
COST_PER_UNIT_DISPENSED             0.000000
TOTAL_SCRIPTS_FILLED                0.000000
PCT_CHANGE_COST_PER_SCRIPT         14.785085
TOTAL_COST_RANK                     0.000000
COST_PER_USER_RANK                  0.000000
COST_PER_SCRIPT_RANK                0.000000
COST_PER_DAYS_SUPPLY_RANK           0.000000
COST_PER_UNIT_DISPENSED_RANK        0.000000
TOTAL_SCRIPTS_FILLED_RANK           0.000000
PCT_CHANGE_COST_PER_SCRIPT_RANK    14.785085
DAYS_SUPPLY_MEDIAN                  0.000000
DAYS_SUPPL

In [17]:
# Drop rows with ~10% or less missing values and confirm rows were dropped
drug = drug.dropna(axis=0, subset=['DOSAGE_FORM', 'ACTIVE_STRENGTH', 'ACTIVE_STRENGTH_UNIT', 'LABELERNAME', 'LAUNCH_YEAR', 'PRODUCT_NDC'])
drug.isnull().sum()* 100 / len(drug)

YEAR                       0.0
RECORD_TYPE                0.0
NUMBER_SCRIPTS             0.0
NDC9                       0.0
PRODUCT_NDC                0.0
LAUNCH_YEAR                0.0
NPROPNAME                  0.0
PROPNAME                   0.0
GENERIC_DRUG_IND_CODE      0.0
THER_CLASS                 0.0
LABELERNAME                0.0
DOSAGE_FORM                0.0
ACTIVE_STRENGTH            0.0
ACTIVE_STRENGTH_UNIT       0.0
PAYER                      0.0
UNIQUE_MEMBERS             0.0
TOTAL_COST                 0.0
COST_PER_USER              0.0
COST_PER_SCRIPT            0.0
COST_PER_DAYS_SUPPLY       0.0
COST_PER_UNIT_DISPENSED    0.0
TOTAL_SCRIPTS_FILLED       0.0
DAYS_SUPPLY_MEDIAN         0.0
DAYS_SUPPLY_MEAN           0.0
DAYS_SUPPLY_STDDEV         0.0
QUANTITY_MEDIAN            0.0
QUANTITY_MEAN              0.0
QUANTITY_STDDEV            0.0
INSURER_PAID_MEDIAN        0.0
INSURER_PAID_MEAN          0.0
INSURER_PAID_STDDEV        0.0
INSURER_PAID_SUM           0.0
MEMBER_P

## Step 3 - Try to Impute Nulls in Percent Change

In [18]:
# analyze head of dataframe
prescription.head()

,YEAR,RECORD_TYPE,NUMBER_SCRIPTS,NPROPNAME,THER_CLASS,PAYER,UNIQUE_MEMBERS,TOTAL_COST,COST_PER_USER,COST_PER_SCRIPT,...,PCT_SCRIPTS_0_18,PCT_SCRIPTS_19_44,PCT_SCRIPTS_45_64,PCT_SCRIPTS_65_PLUS,PCT_SCRIPTS_FEMALE,PCT_SCRIPTS_MALE,PCT_URBAN_CORE,PCT_SUBURBAN,PCT_MICROPOLITAN,PCT_RURAL_SMALLTOWN
0,2012,Drug,137,".ALPHA.,TOCOPHEROL ACETATE, DL,, ASCORBIC ACID...",Nutritional Products,Commercial,99,2228.81,22.513232,16.268686,...,100.0,0.0,0.0,0.0,48.905109,51.094891,20.437956,21.167883,45.255474,13.138686
1,2012,Drug,25,".ALPHA.,TOCOPHEROL ACETATE, DL,, ASCORBIC ACID...",Nutritional Products,Minnesota Health Care Programs,13,219.04,16.849231,8.761600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012,Drug,36,".BETA.,CAROTENE, ASCORBIC ACID, CHOLECALCIFERO...",Nutritional Products,Commercial,26,6430.99,247.345769,178.638611,...,0.0,100.0,0.0,0.0,100.000000,0.000000,NaN,NaN,NaN,NaN
3,2012,Drug,210,".BETA.,CAROTENE, ASCORBIC ACID, CHOLECALCIFERO...",Nutritional Products,Commercial,92,3810.43,41.417717,18.144905,...,NaN,NaN,NaN,NaN,100.000000,0.000000,NaN,NaN,NaN,NaN
4,2012,Drug,117,".BETA.,CAROTENE, ASCORBIC ACID, CHOLECALCIFERO...",Nutritional Products,Minnesota Health Care Programs,56,1082.49,19.330179,9.252051,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# analyze cost per script rank

scriptRank = prescription[['YEAR','COST_PER_USER_RANK', 'COST_PER_SCRIPT_RANK', 'COST_PER_DAYS_SUPPLY_RANK',
                          'COST_PER_UNIT_DISPENSED_RANK', 'TOTAL_SCRIPTS_FILLED_RANK',
                          'PCT_CHANGE_COST_PER_SCRIPT_RANK']]
scriptRank.head()

,YEAR,COST_PER_USER_RANK,COST_PER_SCRIPT_RANK,COST_PER_DAYS_SUPPLY_RANK,COST_PER_UNIT_DISPENSED_RANK,TOTAL_SCRIPTS_FILLED_RANK,PCT_CHANGE_COST_PER_SCRIPT_RANK
0,2012,1319,1241,1416,1208,1142,818.0
1,2012,1156,1149,1199,926,1257,59.0
2,2012,644,456,703,584,1399,NaN
3,2012,1151,1215,1362,1126,1063,164.0
4,2012,1117,1130,1155,895,983,448.0


Appears the 'PCT_CHANGE_COST_PER_SCRIPT_RANK' and 'PCT_CHANGE_COST_PER_SCRIPT' are by year. Since I do not have the prior years (2011 and 2015), I cannot calculate the missing values.

Instead I will make a new feature to compare the growth between 2012 and 2016. These will replace the 'PCT_CHANGE_COST_PER_SCRIPT_RANK' and 'PCT_CHANGE_COST_PER_SCRIPT' features. If the year is 2012, the percent change will be 0. If the year is 2016 I will calculate the percent change as follows:

$PCT\_CHANGE\_COST\_PER\_SCRIPT\_RANK = \frac{(COST\_PER\_SCRIPT\_RANK\_2016 - COST\_PER\_SCRIPT\_RANK\_2012)}{COST\_PER\_SCRIPT\_RANK\_2012} \times 100\%$

$PCT\_CHANGE\_COST\_PER\_SCRIPT = \frac{(COST\_PER\_SCRIPT\_2016 - COST\_PER\_SCRIPT\_2012)}{COST\_PER\_SCRIPT\_2012} \times 100\%$

These formulas will be applied later.

In [20]:
# remove original percent change features and confirm removal
prescription = prescription.drop(['PCT_CHANGE_COST_PER_SCRIPT_RANK','PCT_CHANGE_COST_PER_SCRIPT'], axis = 1)
prescription.isnull().sum()* 100 / len(prescription)

YEAR                             0.000000
RECORD_TYPE                      0.000000
NUMBER_SCRIPTS                   0.000000
NPROPNAME                        0.000000
THER_CLASS                       0.000000
PAYER                            0.000000
UNIQUE_MEMBERS                   0.000000
TOTAL_COST                       0.000000
COST_PER_USER                    0.000000
COST_PER_SCRIPT                  0.000000
COST_PER_DAYS_SUPPLY             0.000000
COST_PER_UNIT_DISPENSED          0.000000
TOTAL_SCRIPTS_FILLED             0.000000
TOTAL_COST_RANK                  0.000000
COST_PER_USER_RANK               0.000000
COST_PER_SCRIPT_RANK             0.000000
COST_PER_DAYS_SUPPLY_RANK        0.000000
COST_PER_UNIT_DISPENSED_RANK     0.000000
TOTAL_SCRIPTS_FILLED_RANK        0.000000
DAYS_SUPPLY_MEDIAN               0.000000
DAYS_SUPPLY_MEAN                 0.000000
DAYS_SUPPLY_STDDEV               0.000000
QUANTITY_MEDIAN                  0.000000
QUANTITY_MEAN                    0

## Step 4 - Properly Join the Two Files

In [21]:
# divide the prescription file into their two years
prescription_2012 = prescription.loc[prescription['YEAR'] == 2012]
prescription_2012 = prescription_2012.add_suffix('_2012')
prescription_2016 = prescription.loc[prescription['YEAR'] == 2016]
prescription_2016 = prescription_2016.add_suffix('_2016')

In [22]:
# join prescription file on ['RECORD_TYPE','NPROPNAME', 'THER_CLASS','PAYER']
prescription_merged = prescription_2012.merge(prescription_2016, how = "outer", left_on = ['RECORD_TYPE_2012','NPROPNAME_2012', 'THER_CLASS_2012','PAYER_2012'],
                                             right_on = ['RECORD_TYPE_2016','NPROPNAME_2016', 'THER_CLASS_2016','PAYER_2016'])
prescription_merged.head()

,YEAR_2012,RECORD_TYPE_2012,NUMBER_SCRIPTS_2012,NPROPNAME_2012,THER_CLASS_2012,PAYER_2012,UNIQUE_MEMBERS_2012,TOTAL_COST_2012,COST_PER_USER_2012,COST_PER_SCRIPT_2012,...,PCT_SCRIPTS_0_18_2016,PCT_SCRIPTS_19_44_2016,PCT_SCRIPTS_45_64_2016,PCT_SCRIPTS_65_PLUS_2016,PCT_SCRIPTS_FEMALE_2016,PCT_SCRIPTS_MALE_2016,PCT_URBAN_CORE_2016,PCT_SUBURBAN_2016,PCT_MICROPOLITAN_2016,PCT_RURAL_SMALLTOWN_2016
0,2012.0,Drug,137.0,".ALPHA.,TOCOPHEROL ACETATE, DL,, ASCORBIC ACID...",Nutritional Products,Commercial,99.0,2228.81,22.513232,16.268686,...,NaN,NaN,NaN,NaN,45.628998,54.371002,9.275053,29.424307,25.799574,35.501066
1,2012.0,Drug,25.0,".ALPHA.,TOCOPHEROL ACETATE, DL,, ASCORBIC ACID...",Nutritional Products,Minnesota Health Care Programs,13.0,219.04,16.849231,8.761600,...,NaN,NaN,NaN,NaN,43.280632,56.719368,11.758893,14.130435,23.023715,51.086957
2,2012.0,Drug,36.0,".BETA.,CAROTENE, ASCORBIC ACID, CHOLECALCIFERO...",Nutritional Products,Commercial,26.0,6430.99,247.345769,178.638611,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012.0,Drug,210.0,".BETA.,CAROTENE, ASCORBIC ACID, CHOLECALCIFERO...",Nutritional Products,Commercial,92.0,3810.43,41.417717,18.144905,...,NaN,NaN,NaN,NaN,100.000000,0.000000,NaN,NaN,NaN,NaN
4,2012.0,Drug,117.0,".BETA.,CAROTENE, ASCORBIC ACID, CHOLECALCIFERO...",Nutritional Products,Minnesota Health Care Programs,56.0,1082.49,19.330179,9.252051,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# lost a significant portion of data with this merge (lost almost 40%)
# additionally, multiple NaN were introduced, but there is a business interpretation to this
# if the NaN is in 2012, then a new drug could have been created and then prescribed in 2016
# if the NaN is in 2016, then a drug is no longer being prescribed that was once available
prescription_merged.shape

(4971, 96)

Upon inspection, it is possible to join the two prescription files, but it will take a lot of work. For example in the 'NPROPNAME' an item has been listed two different ways: CALCIUM PANTOTHEN and CALCIUM P. I believe these are the same but will need outsider information to confirm. Until then these tables will not be joined.

Confirmed this fact on my own using https://www.drugbank.ca/salts/DBSALT000034 and https://www.drugs.com/international/calcium-p.html

Would love if someone else could confirm

In [24]:
# create new percent change columns filled with zeros and check
prescription_merged['PCT_CHANGE_COST_PER_SCRIPT_RANK'] = (prescription_merged['COST_PER_SCRIPT_RANK_2016'] - prescription_merged['COST_PER_SCRIPT_RANK_2012'] ) / prescription_merged['COST_PER_SCRIPT_RANK_2012'] * 100
prescription_merged['PCT_CHANGE_COST_PER_SCRIPT'] = (prescription_merged['COST_PER_SCRIPT_2016'] - prescription_merged['COST_PER_SCRIPT_2012'] ) / prescription_merged['COST_PER_SCRIPT_2012'] * 100
prescription_merged.head()

,YEAR_2012,RECORD_TYPE_2012,NUMBER_SCRIPTS_2012,NPROPNAME_2012,THER_CLASS_2012,PAYER_2012,UNIQUE_MEMBERS_2012,TOTAL_COST_2012,COST_PER_USER_2012,COST_PER_SCRIPT_2012,...,PCT_SCRIPTS_45_64_2016,PCT_SCRIPTS_65_PLUS_2016,PCT_SCRIPTS_FEMALE_2016,PCT_SCRIPTS_MALE_2016,PCT_URBAN_CORE_2016,PCT_SUBURBAN_2016,PCT_MICROPOLITAN_2016,PCT_RURAL_SMALLTOWN_2016,PCT_CHANGE_COST_PER_SCRIPT_RANK,PCT_CHANGE_COST_PER_SCRIPT
0,2012.0,Drug,137.0,".ALPHA.,TOCOPHEROL ACETATE, DL,, ASCORBIC ACID...",Nutritional Products,Commercial,99.0,2228.81,22.513232,16.268686,...,NaN,NaN,45.628998,54.371002,9.275053,29.424307,25.799574,35.501066,0.805802,-21.689198
1,2012.0,Drug,25.0,".ALPHA.,TOCOPHEROL ACETATE, DL,, ASCORBIC ACID...",Nutritional Products,Minnesota Health Care Programs,13.0,219.04,16.849231,8.761600,...,NaN,NaN,43.280632,56.719368,11.758893,14.130435,23.023715,51.086957,-27.502176,183.966173
2,2012.0,Drug,36.0,".BETA.,CAROTENE, ASCORBIC ACID, CHOLECALCIFERO...",Nutritional Products,Commercial,26.0,6430.99,247.345769,178.638611,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012.0,Drug,210.0,".BETA.,CAROTENE, ASCORBIC ACID, CHOLECALCIFERO...",Nutritional Products,Commercial,92.0,3810.43,41.417717,18.144905,...,NaN,NaN,100.000000,0.000000,NaN,NaN,NaN,NaN,-18.600823,94.776976
4,2012.0,Drug,117.0,".BETA.,CAROTENE, ASCORBIC ACID, CHOLECALCIFERO...",Nutritional Products,Minnesota Health Care Programs,56.0,1082.49,19.330179,9.252051,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.805310,13.946508


In [50]:
# divide the drug file into their two years
drug_2012 = drug.loc[drug['YEAR'] == 2012]
drug_2012 = drug_2012.add_suffix('_2012')
drug_2016 = drug.loc[drug['YEAR'] == 2016]
drug_2016 = drug_2016.add_suffix('_2016')

In [51]:
# join drug file on ['NDC9','PRODUCT_NDC']
drug_merged = drug_2012.merge(drug_2016, how = "outer", left_on = ['NDC9_2012','PRODUCT_NDC_2012', 'RECORD_TYPE_2012','NPROPNAME_2012', 'THER_CLASS_2012','PAYER_2012'],
                                             right_on = ['NDC9_2016','PRODUCT_NDC_2016', 'RECORD_TYPE_2016','NPROPNAME_2016', 'THER_CLASS_2016','PAYER_2016'])
drug_merged.head()

,YEAR_2012,RECORD_TYPE_2012,NUMBER_SCRIPTS_2012,NDC9_2012,PRODUCT_NDC_2012,LAUNCH_YEAR_2012,NPROPNAME_2012,PROPNAME_2012,GENERIC_DRUG_IND_CODE_2012,THER_CLASS_2012,...,INSURER_PAID_STDDEV_2016,INSURER_PAID_SUM_2016,MEMBER_PAID_MEDIAN_2016,MEMBER_PAID_MEAN_2016,MEMBER_PAID_STDDEV_2016,MEMBER_PAID_SUM_2016,TOTAL_PAID_MEDIAN_2016,TOTAL_PAID_MEAN_2016,TOTAL_PAID_STDDEV_2016,TOTAL_PAID_SUM_2016
0,2012.0,NDC,61.0,449461023.0,44946-1023,2011.0,".ALPHA.,TOCOPHEROL ACETATE, DL,, ASCORBIC ACID...",MULTIVITAMIN WITH FLUORIDE,Generic,Nutritional Products,...,7.415849,1821.38,7.86,8.464293,7.962693,3529.61,9.60,12.834005,7.793507,5351.78
1,2012.0,NDC,52.0,449461024.0,44946-1024,2011.0,".ALPHA.,TOCOPHEROL ACETATE, DL,, ASCORBIC ACID...",MULTIVITAMIN WITH FLUORIDE,Generic,Nutritional Products,...,6.659666,1349.89,6.49,8.555174,8.167800,2942.98,10.31,12.481977,7.485317,4293.80
2,2012.0,NDC,24.0,449461025.0,44946-1025,2011.0,".ALPHA.,TOCOPHEROL ACETATE, DL,, ASCORBIC ACID...",MULTIVITAMIN WITH FLUORIDE,Generic,Nutritional Products,...,6.209248,637.60,6.76,9.411751,8.017378,1665.88,10.51,13.020734,7.592045,2304.67
3,2012.0,NDC,25.0,449461024.0,44946-1024,2011.0,".ALPHA.,TOCOPHEROL ACETATE, DL,, ASCORBIC ACID...",MULTIVITAMIN WITH FLUORIDE,Generic,Nutritional Products,...,95.787149,16022.55,0.00,0.331366,1.063510,157.73,5.44,34.028298,95.683146,16197.47
4,2012.0,NDC,5608.0,602580150.0,60258-150,1999.0,1.1% SODIUM FLUORIDE PRESCRIPTION DENTAL CREAM,SF 5000 PLUS,Generic,Topical Products,...,3.377492,7683.29,5.89,6.584231,5.117847,29721.22,6.29,8.293062,4.670203,37434.88


In [52]:
# lost a significant portion of data with this merge (lost almost 25%)
# additionally, multiple NaN were introduced, but there is a business interpretation to this
# if the NaN is in 2012, then a new drug could have been created and then prescribed in 2016
# if the NaN is in 2016, then a drug is no longer being prescribed that was once available
drug_merged.shape

(32964, 80)

## Step 5 - Convert to lists where applicable

In [53]:
# NPROPNAME in drug and prescription
prescription_merged['NPROPNAMES_2012'] = prescription_merged['NPROPNAME_2012'].str.split(",")
drug_merged['NPROPNAMES_2012'] = drug_merged['NPROPNAME_2012'].str.split(",")

prescription_merged['NPROPNAMES_2016'] = prescription_merged['NPROPNAME_2016'].str.split(",")
drug_merged['NPROPNAMES_2016'] = drug_merged['NPROPNAME_2016'].str.split(",")

# DOSAGE_FORM in drug
drug_merged['DOSAGE_FORMS_2012'] = drug_merged['DOSAGE_FORM_2012'].str.split(",")

drug_merged['DOSAGE_FORMS_2016'] = drug_merged['DOSAGE_FORM_2016'].str.split(",")

# ACTIVE_STRENGTH in drug
drug_merged['ACTIVE_STRENGTHS_2012'] = drug_merged['ACTIVE_STRENGTH_2012'].str.split(";")

drug_merged['ACTIVE_STRENGTHS_2016'] = drug_merged['ACTIVE_STRENGTH_2016'].str.split(";")

# ACTIVE_STRENGTH_UNIT in drug
drug_merged['ACTIVE_STRENGTH_UNITS_2012'] = drug_merged['ACTIVE_STRENGTH_UNIT_2012'].str.split(";")
drug_merged['ACTIVE_STRENGTH_UNITS_2016'] = drug_merged['ACTIVE_STRENGTH_UNIT_2016'].str.split(";")

# drop original columns
prescription_merged.drop(['NPROPNAME_2012', 'NPROPNAME_2016'], axis = 1)
drug_merged.drop(['NPROPNAME_2012', 'NPROPNAME_2016', 'DOSAGE_FORM_2012', 'DOSAGE_FORM_2016', 
                 'ACTIVE_STRENGTH_2012', 'ACTIVE_STRENGTH_2016', 'ACTIVE_STRENGTH_UNIT_2012',
                 'ACTIVE_STRENGTH_UNIT_2016'], axis = 1)

,YEAR_2012,RECORD_TYPE_2012,NUMBER_SCRIPTS_2012,NDC9_2012,PRODUCT_NDC_2012,LAUNCH_YEAR_2012,PROPNAME_2012,GENERIC_DRUG_IND_CODE_2012,THER_CLASS_2012,LABELERNAME_2012,...,TOTAL_PAID_STDDEV_2016,TOTAL_PAID_SUM_2016,NPROPNAMES_2012,NPROPNAMES_2016,DOSAGE_FORMS_2012,DOSAGE_FORMS_2016,ACTIVE_STRENGTHS_2012,ACTIVE_STRENGTHS_2016,ACTIVE_STRENGTH_UNITS_2012,ACTIVE_STRENGTH_UNITS_2016
0,2012.0,NDC,61.0,449461023.0,44946-1023,2011.0,MULTIVITAMIN WITH FLUORIDE,Generic,Nutritional Products,SANCILIO,...,7.793507,5351.78,"[.ALPHA., TOCOPHEROL ACETATE, DL, , ASCORBIC...","[.ALPHA., TOCOPHEROL ACETATE, DL, , ASCORBIC...","[TABLET, CHEWABLE]","[TABLET, CHEWABLE]","[15, 60, 4.5, .25, .3, 13.5, 1.05, 1.2,...","[15, 60, 4.5, .25, .3, 13.5, 1.05, 1.2,...","[[IU]/1, MG/1, UG/1, MG/1, MG/1, MG/1, M...","[[IU]/1, MG/1, UG/1, MG/1, MG/1, MG/1, M..."
1,2012.0,NDC,52.0,449461024.0,44946-1024,2011.0,MULTIVITAMIN WITH FLUORIDE,Generic,Nutritional Products,SANCILIO,...,7.485317,4293.80,"[.ALPHA., TOCOPHEROL ACETATE, DL, , ASCORBIC...","[.ALPHA., TOCOPHEROL ACETATE, DL, , ASCORBIC...","[TABLET, CHEWABLE]","[TABLET, CHEWABLE]","[15, 60, 4.5, .5, .3, 13.5, 1.05, 1.2, ...","[15, 60, 4.5, .5, .3, 13.5, 1.05, 1.2, ...","[[IU]/1, MG/1, UG/1, MG/1, MG/1, MG/1, M...","[[IU]/1, MG/1, UG/1, MG/1, MG/1, MG/1, M..."
2,2012.0,NDC,24.0,449461025.0,44946-1025,2011.0,MULTIVITAMIN WITH FLUORIDE,Generic,Nutritional Products,SANCILIO,...,7.592045,2304.67,"[.ALPHA., TOCOPHEROL ACETATE, DL, , ASCORBIC...","[.ALPHA., TOCOPHEROL ACETATE, DL, , ASCORBIC...","[TABLET, CHEWABLE]","[TABLET, CHEWABLE]","[15, 60, 4.5, 1, .3, 13.5, 1.05, 1.2, ...","[15, 60, 4.5, 1, .3, 13.5, 1.05, 1.2, ...","[[IU]/1, MG/1, UG/1, MG/1, MG/1, MG/1, M...","[[IU]/1, MG/1, UG/1, MG/1, MG/1, MG/1, M..."
3,2012.0,NDC,25.0,449461024.0,44946-1024,2011.0,MULTIVITAMIN WITH FLUORIDE,Generic,Nutritional Products,SANCILIO,...,95.683146,16197.47,"[.ALPHA., TOCOPHEROL ACETATE, DL, , ASCORBIC...","[.ALPHA., TOCOPHEROL ACETATE, DL, , ASCORBIC...","[TABLET, CHEWABLE]","[TABLET, CHEWABLE]","[15, 60, 4.5, .5, .3, 13.5, 1.05, 1.2, ...","[15, 60, 4.5, .5, .3, 13.5, 1.05, 1.2, ...","[[IU]/1, MG/1, UG/1, MG/1, MG/1, MG/1, M...","[[IU]/1, MG/1, UG/1, MG/1, MG/1, MG/1, M..."
4,2012.0,NDC,5608.0,602580150.0,60258-150,1999.0,SF 5000 PLUS,Generic,Topical Products,CYPRESS PHARMACEUTICAL,...,4.670203,37434.88,[1.1% SODIUM FLUORIDE PRESCRIPTION DENTAL CREAM],[1.1% SODIUM FLUORIDE PRESCRIPTION DENTAL CREAM],[CREAM],[CREAM],[11],[11],[MG/G],[MG/G]
5,2012.0,NDC,3309.0,602580150.0,60258-150,1999.0,SF 5000 PLUS,Generic,Topical Products,CYPRESS PHARMACEUTICAL,...,5.415955,18874.50,[1.1% SODIUM FLUORIDE PRESCRIPTION DENTAL CREAM],[1.1% SODIUM FLUORIDE PRESCRIPTION DENTAL CREAM],[CREAM],[CREAM],[11],[11],[MG/G],[MG/G]
6,2012.0,NDC,2861.0,602580150.0,60258-150,1999.0,SF 5000 PLUS,Generic,Topical Products,CYPRESS PHARMACEUTICAL,...,2.022142,32295.18,[1.1% SODIUM FLUORIDE PRESCRIPTION DENTAL CREAM],[1.1% SODIUM FLUORIDE PRESCRIPTION DENTAL CREAM],[CREAM],[CREAM],[11],[11],[MG/G],[MG/G]
7,2012.0,NDC,179.0,3784105.0,0378-4105,2012.0,ABACAVIR SULFATE,Generic,Anti-Infective Agents,MYLAN,...,NaN,NaN,[ABACAVIR SULFATE],NaN,"[TABLET, FILM COATED]",NaN,[300],NaN,[MG/1],NaN
8,2012.0,NDC,282.0,497020221.0,49702-221,1998.0,ZIAGEN,Brand,Anti-Infective Agents,VIIV HEALTHCARE COMPANY,...,NaN,NaN,[ABACAVIR SULFATE],NaN,"[TABLET, FILM COATED]",NaN,[300],NaN,[MG/1],NaN
9,2012.0,NDC,142.0,3784105.0,0378-4105,2012.0,ABACAVIR SULFATE,Generic,Anti-Infective Agents,MYLAN,...,NaN,NaN,[ABACAVIR SULFATE],NaN,"[TABLET, FILM COATED]",NaN,[300],NaN,[MG/1],NaN


## Step 6 - Write cleaned data to new files

In [54]:
drug_merged.to_excel("C:/Users/LMoor/Downloads/Drug_Clean_v2.xlsx")
prescription_merged.to_excel("C:/Users/LMoor/Downloads/Prescription_Clean_v2.xlsx")